<a href="https://colab.research.google.com/github/AstraBert/DebateLLM-Championship/blob/main/MotionCategoriesAssociations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install necessary dependencies

In [41]:
! pip install --quiet datasets

In [ ]:
! pip install --quiet langchain langchain_core langchain_openai langchain_community

# 2. Download and process necessary data

In [42]:

from datasets import load_dataset

dataset = load_dataset("as-cle-bert/DebateLLMs")
df = dataset["train"]

DebateLLMs_v010.csv:   0%|          | 0.00/68.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20 [00:00<?, ? examples/s]

In [43]:

import pandas as pd

df = pd.DataFrame(df)

# 3. Manually annotate the motions and export the data

In [ ]:
df["motions"].to_list()

In [45]:
annotations = ["Prisoners Extradition", "Oppose Chinese censorship", "Democratization of UN", "Non-violent movements not leading social change", "West funding a coup in Myanmar", "Stop to Bullfighting", "Paper is better than Internet", "Ban to self-diagnose websites", "Public workers have right to strike", "Hedge funds not purchasing sovereign debt", "Trade Unions slow progress", "Cancel 3rd World Debt", "Deny terminally ill patients cures", "Prioritized skilled refugees to enter EU", "Repatriate North Korean refugees", "Not replace workers with technology", "Two parliaments: politicians and experts", "Handmade gifts better than brand gifts", "Do not entrap pedophiles", "Home-country trials for Guantanamo detainees"]

In [46]:
df = df.drop(columns=["Topic"])

In [47]:
df.insert(1, "Topic", annotations)

In [48]:
df.to_csv("DebateLLMs_v011.csv", index=False)

# 4. Extract topics for various settings

In [49]:
df_qwq = df[df["judge"] == "Qwen/QwQ-32B-Preview"]

In [50]:
df_llama = df[df["judge"] == "meta-llama/Llama-3.3-70B-Instruct"]

In [51]:
df_phi = df[df["winner"] == "microsoft/Phi-3.5-mini-instruct"]

In [52]:
df_starchat = df[(df["winner"] != "HuggingFaceH4/starchat2-15b-v0.1") & ((df["favor_model"] == "HuggingFaceH4/starchat2-15b-v0.1") | (df["against_model"] == "HuggingFaceH4/starchat2-15b-v0.1"))]

In [53]:
def extract_topics_for_judges(judges_df):
  judge = judges_df["judge"].to_list()[0]
  topics = {judge: {"Against": [], "Favor": []}}
  for index, row in judges_df.iterrows():
    if row["winner"] == row["favor_model"]:
      topics[judge]["Favor"].append(row["Topic"])
    else:
      topics[judge]["Against"].append(row["Topic"])
  return topics

In [54]:
topics_qwq = extract_topics_for_judges(df_qwq)
topics_llama = extract_topics_for_judges(df_llama)
print(topics_qwq)
print(topics_llama)

{'Qwen/QwQ-32B-Preview': {'Against': ['Prisoners Extradition', 'Non-violent movements not leading social change', 'West funding a coup in Myanmar', 'Paper is better than Internet', 'Public workers have right to strike'], 'Favor': ['Oppose Chinese censorship', 'Democratization of UN', 'Stop to Bullfighting', 'Ban to self-diagnose websites', 'Hedge funds not purchasing sovereign debt']}}
{'meta-llama/Llama-3.3-70B-Instruct': {'Against': ['Deny terminally ill patients cures', 'Prioritized skilled refugees to enter EU', 'Repatriate North Korean refugees'], 'Favor': ['Trade Unions slow progress', 'Cancel 3rd World Debt', 'Not replace workers with technology', 'Two parliaments: politicians and experts', 'Handmade gifts better than brand gifts', 'Do not entrap pedophiles', 'Home-country trials for Guantanamo detainees']}}


In [55]:
def extract_topics_args(df, debater):
  topics = {debater: {"Against": [], "Favor": []}}
  for index, row in df.iterrows():
    if debater == row["favor_model"]:
      topics[debater]["Favor"].append(row["Topic"])
    else:
      topics[debater]["Against"].append(row["Topic"])
  return topics


In [57]:
topics_phi = extract_topics_args(df_phi, "microsoft/Phi-3.5-mini-instruct")
topics_starchat = extract_topics_args(df_starchat, "HuggingFaceH4/starchat2-15b-v0.1")

print(topics_phi)
print(topics_starchat)

{'microsoft/Phi-3.5-mini-instruct': {'Against': ['West funding a coup in Myanmar', 'Repatriate North Korean refugees'], 'Favor': ['Ban to self-diagnose websites', 'Handmade gifts better than brand gifts', 'Do not entrap pedophiles']}}
{'HuggingFaceH4/starchat2-15b-v0.1': {'Against': ['Democratization of UN', 'Stop to Bullfighting', 'Ban to self-diagnose websites', 'Not replace workers with technology', 'Handmade gifts better than brand gifts'], 'Favor': []}}


# 5. Analyze reasons to find "personal opinion" influence

In [60]:
reasons = df["reasons"].to_list()

In [63]:
from google.colab import userdata
import os

a = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = a

In [65]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

GPT_MODEL = "gpt-4o-mini"
llm = ChatOpenAI(temperature=0, model=GPT_MODEL)
system_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a helpful assistant. Your job is to read the verdict from a debate competition and try to find if it was influenced by the personal opinion of the judge. Your analysis output must this way:
            - influence: True if the verdict was influenced by the personal opinion of the judge, False if it wasn't
            - quotes: Quote the sentences where the judge expressed their personal opinion.
            Adhere to the information you are provided with in the verdict, do not add/make up any information."""),
        ("human", "{message}"),
    ]
)

In [66]:
from pydantic import BaseModel, Field

class Analysis(BaseModel):
    """Structure of the output of a debate competition virdict"""
    influence: bool = Field(description="True if the verdict was influenced by the personal opinion of the judge, False if it wasn't")
    quotes: list = Field(description="Quote the sentences where the judge expressed their personal opinion.")

chain = system_prompt | llm.with_structured_output(Analysis)

In [67]:
pseudo_df = {"Influenced": [], "Quotes": []}
for reason in reasons:
  response = chain.invoke({"message": reason})
  pseudo_df["Influenced"].append(response.influence)
  pseudo_df["Quotes"].append("\n\n".join(response.quotes))

if_df = pd.DataFrame.from_dict(pseudo_df)

In [69]:
if_df.insert(0, "Judge", df["judge"].to_list())

In [70]:
if_df.insert(1, "Topic", df["Topic"].to_list())

In [72]:
if_df.insert(2, "Position", ["Against" if row["against_model"] == row["winner"] else "Favor" for index, row in df.iterrows()])

In [74]:
if_df.to_csv("influence1.csv", index=False)

# 6. Perform political compass test

In [75]:
political_compass = load_dataset("happybusinessperson/political-compass")
compass_questions = political_compass["train"]["Question"]
actual_questions = [question.replace("<mask>", "") for question in compass_questions]

political-compass.csv:   0%|          | 0.00/5.41k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/61 [00:00<?, ? examples/s]

In [76]:
question_prompts = "\n\n-----------------------\n\n".join(actual_questions)

In [90]:
from huggingface_hub import InferenceClient
from google.colab import userdata
import json

hf_token = userdata.get('HF_TOKEN_INFERENCE')


client = InferenceClient(api_key=hf_token)

messages = [
	  {"role": "user", "content": f"Build a markdown table in which you report your personal answers to the following questions:\n\n{actual_questions}\n\nTo determine you answer, choose between 'Strongly disagree', 'Disagree', 'Agree', 'Strongly Agree'."},
  ]

completion_llama = client.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct",
  	messages=messages,
  	temperature=0,
  	max_tokens=2048,
  	top_p=0.7
  )

response_llama = completion_llama.choices[0].message.content
print(response_llama)

In [ ]:
completion_phi = client.chat.completions.create(
    model="microsoft/Phi-3.5-mini-instruct",
  	messages=messages,
  	temperature=0,
  	max_tokens=2048,
  	top_p=0.7
  )

response_phi = completion_llama.choices[0].message.content
print(response_phi)

In [ ]:
completion_starchat = client.chat.completions.create(
    model="HuggingFaceH4/starchat2-15b-v0.1",
  	messages=messages,
  	temperature=0,
  	max_tokens=2048,
  	top_p=0.7
  )

response_starchat = completion_llama.choices[0].message.content
print(response_phi)